IMPORT PACKAGES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/home/isaure/.pyenv/versions/3.10.6/envs/fake-news/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nltk
import string

IMPORT DATASET

In [3]:
dataset = load_dataset("mediabiasgroup/mbib-base", "hate-speech")
df= pd.DataFrame(dataset['train'])


Found cached dataset mbib-base (/home/isaure/.cache/huggingface/datasets/mediabiasgroup___mbib-base/hate-speech/1.0.0/cf6f80c612f1363f2162f92f58e1113915a6b01aa07680513a18b7d94570e875)
100%|██████████| 1/1 [00:00<00:00, 219.01it/s]


DEFINE X AND Y 

In [18]:
X_full = df.text
y_full = df.label
X, _, y, _ = train_test_split(X_full, y_full, test_size=0.75, random_state=42)



In [20]:
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def cleaning(series):
    def cleaning_sentence(sentence):
        """ takes a sentence (string) as input and returns
        same string but fully cleaned """
        
        # Basic cleaning
        sentence = sentence.strip() ## remove whitespaces
        sentence = sentence.lower() ## lowercase 
        sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
        
        # Advanced cleaning
        for punctuation in string.punctuation:
            sentence = sentence.replace(punctuation, '') ## remove punctuation
        
        tokenized_sentence = word_tokenize(sentence) ## tokenize 
        
        stop_words = set(stopwords.words('english')) ## define stopwords
        
        tokenized_sentence_cleaned = [ ## remove stopwords
            w for w in tokenized_sentence if not w in stop_words
        ]
        
        # Lemmatizing
        lemmatized_verbs = [WordNetLemmatizer().lemmatize(word, pos = "v") for word in tokenized_sentence_cleaned]
        lemmatized_nouns = [WordNetLemmatizer().lemmatize(word, pos = "n") for word in lemmatized_verbs]
        lemmatized_adj = [WordNetLemmatizer().lemmatize(word, pos = "a") for word in lemmatized_nouns]
        lemmatized_adv = [WordNetLemmatizer().lemmatize(word, pos = "r") for word in lemmatized_adj]
        
        cleaned_sentence = ' '.join(word for word in lemmatized_adv)
        
        return cleaned_sentence
    
    return series.apply(cleaning_sentence)

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [117]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1)
!pip install ngram
import nltk
from nltk.util import ngrams
import tensorflow as tf



2023-06-06 16:40:42.725423: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 16:40:42.740385: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 16:40:42.809319: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 16:40:42.817941: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 16:40:43.935736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn import set_config; set_config("diagram")

#ngrams ((1,1), (2,2), (3,3), (2,3)),
#min_df: 5; 10; 30; 50
#max_df: 0.75; 0.85; 0.95
#nalpha: 0.01; 0.1; 1; 10

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC()),
])

# Set parameters to search
parameters = {
    'tfidf__ngram_range': ((1,1), (1,2), (2,1), (2,3)),
    'tfidf__min_df': (5,10,15, 20,30,40),
    'tfidf__max_df': (0.85,0.9, 0.95),
}

# Perform grid search on pipeline
grid_search = GridSearchCV(
    pipeline, parameters, n_jobs=-1, 
    verbose=1, scoring = "accuracy", 
    cv=4
)

grid_search.fit(X,y)


Fitting 4 folds for each of 72 candidates, totalling 288 fits


/home/isaure/.pyenv/versions/3.10.6/envs/fake-news/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
96 fits failed out of a total of 288.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/home/isaure/.pyenv/versions/3.10.6/envs/fake-news/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/isaure/.pyenv/versions/3.10.6/envs/fake-news/lib/python3.10/site-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/isaure/.pyenv/versions/3.10.6/envs/fake-news/lib/

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('svm', SVC())]),
             n_jobs=-1,
             param_grid={'tfidf__max_df': (0.85, 0.9, 0.95),
                         'tfidf__min_df': (5, 10, 15, 20, 30, 40),
                         'tfidf__ngram_range': ((1, 1), (1, 2), (2, 1),
                                                (2, 3))},
             scoring='accuracy', verbose=1)

In [101]:
best_model = grid_search.best_estimator_
best_params = best_model.get_params()
best_score = grid_search.best_score_
print(f"best ngram:{best_params['tfidf__ngram_range']} ; best mindf: {best_params['tfidf__min_df']}; best maxdf: {best_params['tfidf__max_df']}")
#wiht an accuracy score of {best_score}")

best ngram:(1, 2) ; best mindf: 10; best maxdf: 0.85


In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [103]:
from sklearn.preprocessing import FunctionTransformer

cleaner = FunctionTransformer(cleaning)

In [104]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.85, ngram_range=(1, 2))

In [108]:
model = ANN(kernel='linear', C=2)

NameError: name 'ANN' is not defined

In [106]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(cleaner, vectorizer, model)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function cleaning at 0x7f4b48e0b910>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.85, min_df=10, ngram_range=(1, 2))),
                ('svc', SVC(C=2, kernel='linear'))])

In [107]:
from sklearn.metrics import accuracy_score

predictions = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


train_features = vectorizer.fit_transform(X_train)
test_features = vectorizer.transform(X_test)